# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [67]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [17]:
html_page = requests.get('http://books.toscrape.com/catalogue/page-1.html')
soup = BeautifulSoup(html_page.content)

In [92]:
soup.prettify()


'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="no-js" lang="en-us">\n <!--<![endif]-->\n <head>\n  <title>\n   All products | Books to Scrape - Sandbox\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>\n  <meta content="24th Jun 2016 09:33" name="created"/>\n  <meta content="" name="description"/>\n  <meta content="width=device-width" name="viewport"/>\n  <meta content="NOARCHIVE,NOCACHE" name="robots"/>\n  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n  <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n  <link href="../static/oscar/favicon.ico" rel="shortcut icon"/>\n  <link href="../static/oscar/css/styles.css

In [33]:
def retrieve_titles(soup):
    title_list = [title_h.find(title=True).attrs['title'] 
              for title_h in soup.find_all('article', class_ = 'product_pod')]
    return title_list

retrieve_titles(soup)

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [48]:
def retrieve_ratings(soup):
    rating_dict = {'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}
    
    rating_list = [rating_dict[rating.attrs['class'][1]] 
              for rating 
              in soup.find_all('p', class_ = re.compile('star-rating *'))]
    
    return rating_list

retrieve_ratings(soup)

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [60]:
def retrieve_prices(soup):
    price_list = [float(price.string[1:]) for price in soup.find_all('p', class_ = 'price_color')]
    return price_list

retrieve_prices(soup)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [66]:
def retrieve_availabilities(soup):
    avail_list = [avail.contents[2].strip() for avail in soup.find_all('p', class_ = 'availability')]
    return avail_list

retrieve_availabilities(soup)

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [91]:
df = pd.DataFrame()

for i in range(1,51):
    url = f"http://books.toscrape.com/catalogue/page-{i}.html"
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content)
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_availabilities(soup)

    columns = ['Title', 'Rating', 'Price', 'Availability']
    col_lists = [new_titles, new_star_ratings, new_prices, new_avails]
    
    if len(df) == 0:
        # insert first page's results
        for x, c in enumerate(columns):
            df[c] = col_lists[x]
    else:
        # Insert supplemental page results in a new dataframe and combine
        df2 = pd.DataFrame()
        for x, c in enumerate(columns):
            df2[c] = col_lists[x]
        df = df.append(df2, ignore_index=True)
        
df

,Title,Rating,Price,Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.10,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [98]:
# get first page
url = f"http://books.toscrape.com"
html_page = requests.get(url)
soup = BeautifulSoup(html_page.content)

new_titles = retrieve_titles(soup)
new_star_ratings = retrieve_ratings(soup)
new_prices = retrieve_prices(soup)
new_avails = retrieve_availabilities(soup)

columns = ['Title', 'Rating', 'Price', 'Availability']
col_lists = [new_titles, new_star_ratings, new_prices, new_avails]

# Load initial data into dataframe
df = pd.DataFrame()
for x, c in enumerate(columns):
    df[c] = col_lists[x]

#If there is a next page, loop through
while soup.find('li', class_='next'):
    next_url = soup.find('li', class_='next').find('a').attrs['href']
    html_page = requests.get(url + '/' + next_url)
    soup = BeautifulSoup(html_page.content)
    
    #do something
    print(url + '/' + next_url)

http://books.toscrape.com/catalogue/page-2.html
http://books.toscrape.com/page-3.html


In [97]:
    #extract info from new page
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_availabilities(soup)
    
    # Insert supplemental page results in a new dataframe and combine
    df2 = pd.DataFrame()
    for x, c in enumerate(columns):
        df2[c] = col_lists[x]
    df = df.append(df2, ignore_index=True)

catalogue/page-2.html


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!